In [87]:
import pandas as pd
import numpy as np
import itertools, csv
import uproot, uproot_methods
import uproot_methods.classes.TLorentzVector as TLorentzVector

In [88]:
# *** 0. Import Dataset
qcd_raw = pd.read_csv('../higgsReconstruction/EventPlotting/qcd_outputDataForLearning.csv')
hh_raw = pd.read_csv('../higgsReconstruction/EventPlotting/dihiggs_outputDataForLearning.csv')

qcd_raw.head()
print(len(qcd_raw), "rows of qcd data")
hh_raw.head()
print(len(hh_raw), "rows of dihiggs data")
#print(hh_raw.columns)

# *** 1. Make higgs and qcd sets from raw data
nJets = 4
variableNames = []
for jet in range(1, nJets+1):
    variableNames.append('jet'+str(jet)+'_pt')
    variableNames.append('jet'+str(jet)+'_eta')
    variableNames.append('jet'+str(jet)+'_phi')
    variableNames.append('jet'+str(jet)+'_mass')   
#print(variableNames)
jetPairs = list(itertools.combinations( np.arange(1,nJets+1),2))

hh_jetData  = hh_raw[variableNames]
qcd_jetData = qcd_raw[variableNames]

1703 rows of qcd data
4605 rows of dihiggs data


In [167]:
def returnJetsAndDijets1D(_event, _allJetPairs):
    """function to calculate jet pairs and return list"""
    
    _jetsAndDijets = []
    _dijetVectors = {}
    
    for _pair in _allJetPairs:
       
        # make lorentz vectors and dijet
        v_jetA = TLorentzVector.PtEtaPhiMassLorentzVector( _event['jet{0}_pt'.format(_pair[0])], _event['jet{0}_eta'.format(_pair[0])], _event['jet{0}_phi'.format(_pair[0])], _event['jet{0}_mass'.format(_pair[0])])
        v_jetB = TLorentzVector.PtEtaPhiMassLorentzVector( _event['jet{0}_pt'.format(_pair[1])], _event['jet{0}_eta'.format(_pair[1])], _event['jet{0}_phi'.format(_pair[1])], _event['jet{0}_mass'.format(_pair[1])])
        v_dijetAB = v_jetA + v_jetB

        # list of jets and dijets for dijet training
        _jetsAndDijets.append(v_jetA.pt)
        _jetsAndDijets.append(v_jetA.eta)
        _jetsAndDijets.append(v_jetA.phi)
        _jetsAndDijets.append(v_jetA.mass)

        _jetsAndDijets.append(v_jetB.pt)
        _jetsAndDijets.append(v_jetB.eta)
        _jetsAndDijets.append(v_jetB.phi)
        _jetsAndDijets.append(v_jetB.mass)

        _jetsAndDijets.append(v_dijetAB.pt)
        _jetsAndDijets.append(v_dijetAB.eta)
        _jetsAndDijets.append(v_dijetAB.phi)
        _jetsAndDijets.append(v_dijetAB.mass)


        # list of dijet vectors for quadjet training
        #_dijetVectors.append(v_dijetAB)
        _dijetVectors[str(_pair[0])+str(_pair[1])] = v_dijetAB
        
        
    #return list for dijet training and list of vectors for quadjet manipulation
    return _jetsAndDijets, _dijetVectors


def makeDijetLabels(_allJetPairs):
    """function to make new variable header"""
    
    _outputVariablesForJetsAndDijets = []
    _dijetLabels = []
    
    for _pair in _allJetPairs:
        _outputVariablesForJetsAndDijets.append('jet'+str(_pair[0])+'_pt')
        _outputVariablesForJetsAndDijets.append('jet'+str(_pair[0])+'_eta')
        _outputVariablesForJetsAndDijets.append('jet'+str(_pair[0])+'_phi')
        _outputVariablesForJetsAndDijets.append('jet'+str(_pair[0])+'_mass') 
        
        _outputVariablesForJetsAndDijets.append('jet'+str(_pair[1])+'_pt')
        _outputVariablesForJetsAndDijets.append('jet'+str(_pair[1])+'_eta')
        _outputVariablesForJetsAndDijets.append('jet'+str(_pair[1])+'_phi')
        _outputVariablesForJetsAndDijets.append('jet'+str(_pair[1])+'_mass') 
        
        _dijetLabel = str(_pair[0])+str(_pair[1])
        _dijetLabels.append(_dijetLabel)
        _outputVariablesForJetsAndDijets.append('dijet'+_dijetLabel+'_pt')
        _outputVariablesForJetsAndDijets.append('dijet'+_dijetLabel+'_eta')
        _outputVariablesForJetsAndDijets.append('dijet'+_dijetLabel+'_phi')
        _outputVariablesForJetsAndDijets.append('dijet'+_dijetLabel+'_mass') 
        
    return _outputVariablesForJetsAndDijets, _dijetLabels


def returnDijetsAndQuadjets1D(_dijetVectors, _dijetLabels):
    """function to create quadjets from dijet inputs and return list"""
    
    _dijetsAndQuadjets = []
    _quadjetLabels = []
    _outputVariableNamesForDijetsAndQuadjets = []
     
    for _firstLabel in _dijetLabels:
        _jetA = _firstLabel[0]
        _jetB = _firstLabel[1]
        for _secondLabel in _dijetLabels:
            # only look at pairs without repeated jets
            if _jetA in _secondLabel or _jetB in _secondLabel:
                continue
            
            # skip if pair already processed
            if _firstLabel+_secondLabel in _quadjetLabels or _secondLabel+_firstLabel in _quadjetLabels:
                continue
                
            # start processing
      
            _outputVariableNamesForDijetsAndQuadjets.append('dijet'+_firstLabel+'_pt')
            _outputVariableNamesForDijetsAndQuadjets.append('dijet'+_firstLabel+'_eta')
            _outputVariableNamesForDijetsAndQuadjets.append('dijet'+_firstLabel+'_phi')
            _outputVariableNamesForDijetsAndQuadjets.append('dijet'+_firstLabel+'_mass') 
        
            _outputVariableNamesForDijetsAndQuadjets.append('dijet'+_secondLabel+'_pt')
            _outputVariableNamesForDijetsAndQuadjets.append('dijet'+_secondLabel+'_eta')
            _outputVariableNamesForDijetsAndQuadjets.append('dijet'+_secondLabel+'_phi')
            _outputVariableNamesForDijetsAndQuadjets.append('dijet'+_secondLabel+'_mass') 
        
            _quadjetLabel = _firstLabel + _secondLabel
            _quadjetLabels.append(_firstLabel+_secondLabel)
            _outputVariableNamesForDijetsAndQuadjets.append('quadjet'+_quadjetLabel+'_pt')
            _outputVariableNamesForDijetsAndQuadjets.append('quadjet'+_quadjetLabel+'_eta')
            _outputVariableNamesForDijetsAndQuadjets.append('quadjet'+_quadjetLabel+'_phi')
            _outputVariableNamesForDijetsAndQuadjets.append('quadjet'+_quadjetLabel+'_mass') 
        
            # list of dijets and quadjets for quadjet training
            _dijetsAndQuadjets.append(_dijetVectors[_firstLabel].pt)
            _dijetsAndQuadjets.append(_dijetVectors[_firstLabel].eta)
            _dijetsAndQuadjets.append(_dijetVectors[_firstLabel].phi)
            _dijetsAndQuadjets.append(_dijetVectors[_firstLabel].mass)

            _dijetsAndQuadjets.append(_dijetVectors[_secondLabel].pt)
            _dijetsAndQuadjets.append(_dijetVectors[_secondLabel].eta)
            _dijetsAndQuadjets.append(_dijetVectors[_secondLabel].phi)
            _dijetsAndQuadjets.append(_dijetVectors[_secondLabel].mass)

            _quadjet = _dijetVectors[_firstLabel] + _dijetVectors[_secondLabel]
            _dijetsAndQuadjets.append(_quadjet.pt)
            _dijetsAndQuadjets.append(_quadjet.eta)
            _dijetsAndQuadjets.append(_quadjet.phi)
            _dijetsAndQuadjets.append(_quadjet.mass)           
  
    return _dijetsAndQuadjets, _outputVariableNamesForDijetsAndQuadjets


In [170]:
# *** 2. Store 1D jet and dijet data in .csv for later usage
outputVariableNamesForJetsAndDijets, dijetLabels = makeDijetLabels(jetPairs)

csvFileForJetsAndDijets = open('jetAndDijetVectorFor1DResnet.csv', mode='w')
writerForJetsAndDijets = csv.DictWriter(csvFileForJetsAndDijets, fieldnames=outputVariableNamesForJetsAndDijets)
writerForJetsAndDijets.writeheader()

firstEvent = False
for iEvent in range(0, hh_jetData['jet1_pt'].count()):
    listOfJetAndDijetKinematics, dijetVectors = returnJetsAndDijets1D(hh_jet.iloc[iEvent], jetPairs)
    #listOfJetAndDijetKinematics, dijetVectors = returnJetsAndDijets1D(hh_jet.iloc[iEvent], jetPairs)
    listOfDijetAndQuadjetKinematics, outputVariableNamesForDijetsAndQuadjets = returnDijetsAndQuadjets1D(dijetVectors, dijetLabels)

    if firstEvent == False:
        csvFileForDijetsAndQuadjets = open('dijetAndQuadjetVectorFor1DResnet.csv', mode='w')
        writerForDijetsAndQuadjets = csv.DictWriter(csvFileForDijetsAndQuadjets, fieldnames=outputVariableNamesForDijetsAndQuadjets)
        writerForDijetsAndQuadjets.writeheader()
        firstEvent=True
        
    # ** A. Make .csv for jets and dijets
    csvLineDictForJetsAndDijets = {}
    for iVariable in range(0, len(outputVariableNamesForJetsAndDijets)):
        # format csv line using names of variables
        csvLineDictForJetsAndDijets[outputVariableNamesForJetsAndDijets[iVariable]] = listOfJetAndDijetKinematics[iVariable]
        
    # ** B. Make .csv for dijets and quadjets
    csvLineDictForDijetsAndQuadjets = {}
    for iVariable in range(0, len(outputVariableNamesForDijetsAndQuadjets)):
        # format csv line using names of variables
        csvLineDictForDijetsAndQuadjets[outputVariableNamesForDijetsAndQuadjets[iVariable]] = listOfDijetAndQuadjetKinematics[iVariable]
        
    # write lines to .csv
    writerForJetsAndDijets.writerow( csvLineDictForJetsAndDijets )
    writerForDijetsAndQuadjets.writerow( csvLineDictForDijetsAndQuadjets )
    
csvFile.close()


[146.37563519574, -0.8145153799514057, 1.4834960093508884, 148.2918364672722, 44.305394809591306, -2.4768705244114138, 2.8832412353610795, 158.2966459273912, 159.98938434679764, -1.6349107158186393, 1.7598873411091074, 357.8633628294124, 233.56778590677996, -0.7871332043398371, 1.9131856146940218, 93.30537154052674, 79.31098809195137, -1.6138864072385497, -0.9152856042158054, 106.39561062033789, 159.98938434679764, -1.6349107158186393, 1.7598873411091074, 357.8633628294124, 120.2068200709832, -1.7283426707857226, 1.5068742738165575, 220.98612316117166, 52.98334810375416, -1.055573630267523, 2.3638670284214887, 114.84637231564902, 159.98938434679764, -1.6349107158186393, 1.7598873411091074, 357.8633628294124]
[159.55493212693327, -0.3349794144424467, -1.4255564298431818, 96.55363983295942, 127.97305538491788, 1.1668023275872506, 1.2032403998615548, 58.55460134776298, 79.05746716633669, 1.279673881978956, -0.5079058971071015, 398.80733466093403, 76.36737551515752, 1.3437689659580392, -0.

[362.2306170460343, 0.8504679509711883, 1.380297701838724, 109.53803559013004, 107.96916986511867, 1.2683069982957913, 2.3845619276838033, 55.12674765662898, 429.94124441902596, 1.0268851269709394, 1.5938076085107604, 265.52589471928036, 388.1431573643744, 1.0165875076043114, 1.486500420945809, 132.06531890384733, 60.55425482571804, 0.8409897964288799, 2.350476012803168, 66.65631258220148, 429.94124441902596, 1.0268851269709396, 1.5938076085107604, 265.52589471928013, 315.59124148706167, 1.0431361208100223, 1.4423054811654357, 170.14311642329713, 127.2177271044564, 0.9025002450984096, 1.9775549504787147, 71.22515068592887, 429.94124441902596, 1.0268851269709396, 1.5938076085107604, 265.52589471928013]
[70.89908872021951, -2.6389319494142134, -1.5164195913595153, 103.19678459113696, 52.15498197873749, 0.6139177353134319, 0.5516448413579758, 103.4033498065441, 64.9537056461522, -2.654851095177779, -0.7329956620739025, 426.00439692289467, 81.35199106388649, -2.237869636570224, -1.22851535

[58.94939029728323, -1.5554587044734032, -0.635640813781927, 91.6438949807846, 66.88706472646965, -2.297248201562185, 1.8909495401725205, 91.95565290356788, 38.83199751069023, -3.172738479150345, 0.8237306912527333, 235.48216617244074, 48.85229288732116, -2.3600419082719646, -2.454359064988093, 136.0537562187968, 87.48282773768634, -1.5932299412547422, 0.7476709290016494, 31.474720365287894, 38.83199751069023, -3.172738479150345, 0.8237306912527335, 235.48216617244105, 64.32353569809567, -1.921964074983436, -0.544151832855493, 108.51392262546779, 68.10801990890056, -2.0022711772932773, 2.004918009175819, 87.4079350352765, 38.831997510690236, -3.172738479150345, 0.8237306912527332, 235.4821661724408]
[96.55826115642162, 1.398442551870081, 1.4721262259584136, 85.41754955615832, 104.23168566049983, 0.8486865533314991, -2.9218207758993175, 91.06437879900022, 117.83739839976914, 1.610127370753378, 2.469533911862051, 248.67058331448052, 94.43396134730712, 1.6591628687281512, 2.77594751680992

[34.54293618923208, -3.039248759382228, 2.5100260117717035, 236.66485774222713, 70.13075927715816, -1.6878383800500536, -0.656282665699492, 230.47239897977042, 35.6086078315478, -3.418940655939066, -0.6802588602659586, 493.6507166606231, 77.60586073819172, -2.6237050597661677, 1.8899982249822629, 172.08165339020195, 109.26909516952166, -0.10011389333413731, -1.0744478420288381, 129.97372162913868, 35.6086078315478, -3.418940655939066, -0.6802588602659586, 493.6507166606231, 107.8296129974774, -1.545714012026035, 2.4090715621875916, 271.1470589510345, 143.40166719690183, -1.4890563206104523, -0.7195492030736385, 123.33208719645235, 35.60860783154779, -3.4189406559390663, -0.6802588602659583, 493.650716660623]
[133.21942460060993, 0.9303012284454564, -2.7027025237682842, 139.84756406737367, 112.65006597638593, 1.250428999218795, -2.016425299182273, 132.86593966275404, 231.63952019633652, 1.135100164197231, -2.389444283963695, 288.0326789533833, 207.52053561048953, 0.90058660578437, -2.55

[59.22653406546136, 0.959308357885434, -2.05026791276175, 146.67903614575062, 61.678415233128106, 1.3038337159105078, 0.3512853896356989, 153.4296515784658, 43.78302918732415, 2.0724776068562853, -0.7972569074852193, 322.1094748829591, 30.72935951018873, 2.439725399109778, -2.826439345775027, 195.1152095674724, 63.654941542560486, -0.05857072137471184, -0.3495342662110644, 81.599944450034, 43.783029187324146, 2.072477606856286, -0.7972569074852192, 322.10947488295915, 133.5200458089393, 0.255510601915324, -2.0351535759383967, 95.07311543217638, 126.18971886043924, 0.9389140456531057, 0.7723451896900428, 73.13548715226989, 43.78302918732416, 2.0724776068562853, -0.7972569074852193, 322.10947488295915]
[130.68807323495062, 0.053326401221142766, -2.4535463757985485, 120.67670183665943, 102.58333624973105, 0.3085052325128753, 1.0265331348741817, 112.97062596927213, 48.07582438578194, 0.743411618390414, 3.0422060137384115, 328.1637063775933, 31.182663970030895, -0.5198931312529064, -2.29135

...

[217.85653534073288, -0.7611046548476958, -0.191537101204383, 58.765664563771075, 225.09478002548107, 0.1963707082416866, 2.9511903687080396, 117.2304782110673, 7.242605754665011, -3.6396858892307113, 2.985332105055756, 530.0713571995968, 52.45619997469315, -1.7755399394942286, -1.6713722328961131, 419.6065998261512, 53.351642546267335, 0.23384909698011125, 1.6061807495478793, 82.34063966120692, 7.242605754664994, -3.639685889230714, 2.9853321050557557, 530.0713571995968, 150.98603476870576, -0.844389897747105, 0.032296149216934834, 244.84498329490555, 158.10609948577752, 0.03406451498675857, -3.1178830208338075, 161.9783550535654, 7.24260575466501, -3.639685889230712, 2.985332105055757, 530.0713571995968]
[180.34176219865114, -0.49183055646213114, 0.016743474100088883, 74.1327433825327, 35.00789719395029, -0.9483750480227175, -1.5174290350218913, 55.32811889080581, 184.9622698632097, -0.6581924518599241, -0.17354641297405943, 183.72444061442772, 138.29222448046656, -0.5885372796420828

[105.25063731384576, 1.4037727547421812, 1.8717513306999978, 99.76876213515136, 107.61988417716489, 0.9318706993362035, -1.0673195871315824, 94.21389551023987, 21.6473469698674, 3.3774809514935034, 0.2931127468751022, 292.82717083653597, 71.128361605692, 1.9533641980210508, 0.5857689716956399, 228.6077321780627, 50.786882399872546, 1.1363758201747458, -2.432543194936292, 16.38728583233493, 21.6473469698674, 3.3774809514935034, 0.2931127468751025, 292.8271708365361, 109.33990197321916, 1.4842055534968628, 1.9259370230590973, 98.70188246116061, 112.77084632273284, 0.7168877384436476, -1.0228741791195235, 71.29310596056868, 21.6473469698674, 3.3774809514935034, 0.29311274687510225, 292.82717083653597]
[185.69811685895343, -0.33836901959705024, -1.7401471948108627, 110.71830868877784, 136.68843517247348, 0.7624857152893779, 1.6589464089762331, 100.27946538569189, 63.8413607625973, 0.7266157106858191, -2.316838561718658, 422.3425171563686, 90.41802362495885, 0.13205196245294715, -1.06141197

[166.08019876494424, 1.9648334410500037, 1.4158715732116698, 97.64497806396491, 140.33237317456803, 1.0490997602324987, -1.5652197155731813, 114.93334024187455, 35.525376727507904, 3.752151738438791, 0.7326509457822743, 416.77557525609717, 66.55212107194127, 3.086172402217392, 1.1005275126442582, 241.00394447018812, 35.763575592645175, 0.7520073072154991, -1.6757543656495906, 83.66292446363991, 35.5253767275079, 3.7521517384387915, 0.732650945782274, 416.77557525609745, 109.99486049053596, 2.3159024243703588, 0.9948489945309061, 279.92695962526307, 76.24177385203463, 1.7132302740511767, -2.0256701507511146, 89.32386201086202, 35.52537672750789, 3.752151738438792, 0.7326509457822742, 416.77557525609717]
[90.14275537807765, 0.35219759922209454, -0.5308067426217651, 113.7503790683453, 95.17407649882993, -1.6000262291614291, 2.3930601047158313, 110.74908654542718, 20.74621844552104, -2.9299001953766415, 1.174607127299189, 369.6834580037607, 76.20815390984181, 0.06376651454410807, 0.8850671

[52.42087530221632, -1.820142354667303, 3.116014304577487, 110.22863459097817, 15.36029610007568, 0.5680278953936869, -0.10369586904664017, 108.37490391983158, 37.12677798250445, -2.093633862902868, -3.1348790383842897, 268.66911348381353, 24.38449109237674, -2.0986441894332786, -2.939888587104001, 159.45693243939706, 14.024203411837302, -1.9913258080983383, 2.804690173287065, 108.46662237557365, 37.12677798250445, -2.093633862902868, -3.1348790383842897, 268.6691134838136, 131.68714249822978, -0.5804779589456465, -3.053681477978504, 83.05691987918526, 94.7305600580394, -0.6632145105768, 0.11970450279947151, 65.93747353004592, 37.12677798250443, -2.093633862902869, -3.1348790383842897, 268.66911348381353]
[58.216015550434406, 1.2565050671776516, -2.008542012811885, 89.51080028419352, 39.53165771970086, 1.4554395186807687, 0.30621457597670615, 95.53984311761133, 42.83503737848306, 2.110147797638212, -1.2620754817226574, 205.93742523784567, 36.42093649792909, 1.5656106730762955, -0.74083

[78.8558506337841, 1.2203958148747394, -0.3085307041839092, 62.27785161471826, 27.89974517845194, 0.05271729513103687, 0.11781618434289678, 66.91068795599678, 104.89456648261647, 1.001026472994505, -0.1983127678694571, 164.03457114908124, 107.09009987941913, 0.6297145291646551, -0.6198485164816252, 27.921832747888267, 44.40142695655637, 0.9893838450479355, 1.2100876648292247, 71.31823504835997, 104.89456648261647, 1.001026472994505, -0.19831276786945706, 164.03457114908124, 47.764179368813245, 0.8627180268586949, -0.3320876365658658, 96.61299160502458, 57.908623402832, 1.0956524843596496, -0.08807829600411599, 59.43612702609494, 104.89456648261647, 1.001026472994505, -0.1983127678694571, 164.03457114908124]
[111.87917107456616, 0.29016418457756243, -0.036508057526150176, 157.83823293015354, 158.7983422201933, 0.8326744803627171, 1.5379686838801747, 114.7470558106285, 193.91515811130617, 0.833360102171009, 0.9229850346284748, 353.33563851210283, 157.25363791574523, 0.8822475317788478, 0